In [1]:
import numpy as np
import re
import copy
import random

# Part related to the board

In [2]:
'''
0: cell is empty
1: cell is occupied by cross
2: cell is occupied by circle
'''
def init_board():
    return np.zeros((3, 3), dtype=int)
def is_empty(x, y, board):
    return not board[x][y]

def display(board):
    values = list(map(lambda x: ' ' if x==0 else ('\u20dd' if x==2 else '\u2a09') , board.flatten()))
    mystr = '''
    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | {} | {} | {} |
    |---+---+---+---|
    | 1 | {} | {} | {} |
    |---+---+---+---|
    | 2 | {} | {} | {} |
    +---+---+---+---+
    '''.format(*values)
    print(mystr)

def exist_combo(board):
    if np.any(np.sum(board, axis=1) == 3):
        return True
    if np.any(np.sum(board, axis=0) == 3):
        return True
    if np.trace(board) == 3:
        return True
    if np.trace(board.T) == 3:
        return True
    return False


def is_end(board):
    crosses = np.zeros_like(board)
    crosses[board == 1] = 1
    if exist_combo(crosses):
        return True, 1
    circles = np.zeros_like(board)
    circles[board == 2] = 1
    if exist_combo(circles):
        return True, 2
    if np.sum(board > 0)== 9:
        return True, None
    return False, None

def make_move(x, y, board):
    if x < 0 or x > 2 or y < 0 or y > 2:
        print (f'Coordinates ({x}, {y}) are not in the board')
        return False
    if is_empty(x, y, board):
        return True
    print(f'Coordinates ({x}, {y}) are already occupied, change coordinates')
    return False

# Players

In [3]:
def bot_turn(board, turn):
    cell = choose_move(board, turn, turn)
    board[cell[0]][cell[1]] = turn
    print(f'Bot makes move {cell}')
    return board

def player_turn(board, turn):
    while True:
        print('Enter x and y coordinates: x y')
        try:
            raw_inp = input()
            inp = re.split('[, ]+', raw_inp)
            x, y = int(inp[0]), int(inp[1])
        except:
            print(f'Input format {raw_input} is incorect')
            continue
        if make_move(x, y, board):
            board[x][y] = turn
            print("You've made a move")
            break
    return board

# Smart part

In [4]:
def free_cells(board):
    return np.argwhere(board == 0)

def choose_move(board, turn, winner):
    return max_choice(copy.deepcopy(board), turn, winner)

def max_choice(board, turn, winner):
    
    cells = free_cells(board)
    values = []
    for i in cells:
        new_board = copy.deepcopy(board)
        new_board[i[0]][i[1]] = turn
        values.append(min_choice(new_board, 2 - (turn + 1) % 2, winner))
    possible_choices = np.argwhere(values == np.max(values)).flatten()
    cell_ind = random.choice(possible_choices)
    return cells[cell_ind]

def min_choice(board, turn, winner):
    status = is_end(board)
    if status[0]:
        if status[1] == None:
            return 0
        elif status[1] == winner:
            return 1
        else:
            return -1
    cells = free_cells(board)
    values = []
    for i in cells:
        new_board = copy.deepcopy(board)
        new_board[i[0]][i[1]] = turn
        values.append(init_walking(new_board, 2 - (turn + 1) % 2, winner))
    return min(values)
        

def init_walking(board, turn, winner):
    # check if the game is already over
    status = is_end(board)
    if status[0]:
        if status[1] == None:
            return 0
        elif status[1] == winner:
            return 1
        else:
            return -1
    counter = 0
    for i in range(20):
        cells = free_cells(board)
        new_board = copy.deepcopy(board)
        cell = cells[np.random.randint(len(cells))]
        new_board[cell[0]][cell[1]] = turn
        counter += recursive_walk(new_board, 2 - (turn + 1) % 2, winner)
    return counter / 20

def recursive_walk(board, turn, winner):
    
    status = is_end(board)
    if status[0]:
        if status[1] == None:
            return 0
        elif status[1] == winner:
            return 1
        else:
            return -1
        
    cells = free_cells(board)
    new_board = copy.deepcopy(board)
    cell = cells[np.random.randint(len(cells))]
    new_board[cell[0]][cell[1]] = turn 
    return recursive_walk(new_board, 2 - (turn + 1) % 2, winner)

# Game itself

In [5]:
def start_game():
    print('The new game starts. Please choose your turn (1 or 2)')
    while True:
        inp  = input()
        try:
            if int(inp) != 1 and int(inp) != 2:
                print(f'Input {inp} is incorrect. Print 1 or 2')
                continue
        except:
            print(f'Input {inp} is incorrect. Print 1 or 2')
            continue
        turn  = int(inp)
        break
    board = init_board()
    count = 1
    while True:
        display(board)
        if count % 2 == turn % 2:
            board = player_turn(board, 2 - count % 2)
        else:
            board = bot_turn(board, 2 - count % 2)
        game_status = is_end(board)
        if game_status[0]:
            print('The game is over')
            display(board)
            if game_status[1] == None:
                print('It is a draw')
            else:
                print('YOU {}'.format('WON!!!' if count % 2 == turn % 2 else 'LOSE :(())'))
            break
        count += 1
start_game()

The new game starts. Please choose your turn (1 or 2)


 2



    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 |   |   |   |
    |---+---+---+---|
    | 1 |   |   |   |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Bot makes move [1 1]

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 |   |   |   |
    |---+---+---+---|
    | 1 |   | ⨉ |   |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Enter x and y coordinates: x y


 0 0


You've made a move

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ |   |   |
    |---+---+---+---|
    | 1 |   | ⨉ |   |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Bot makes move [1 0]

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ |   |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ |   |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Enter x and y coordinates: x y


...

You've made a move

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ |   |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Bot makes move [0 1]

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ | ⨉ |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 |   |   |   |
    +---+---+---+---+
    
Enter x and y coordinates: x y


 2 1


You've made a move

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ | ⨉ |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 |   | ⃝ |   |
    +---+---+---+---+
    
Bot makes move [2 2]

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ | ⨉ |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 |   | ⃝ | ⨉ |
    +---+---+---+---+
    
Enter x and y coordinates: x y


 2 0


You've made a move

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ | ⨉ |   |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 | ⃝ | ⃝ | ⨉ |
    +---+---+---+---+
    
Bot makes move [0 2]
The game is over

    +---+---+---+---+
    |   | 0 | 1 | 2 |
    |---+---+---+---|
    | 0 | ⃝ | ⨉ | ⨉ |
    |---+---+---+---|
    | 1 | ⨉ | ⨉ | ⃝ |
    |---+---+---+---|
    | 2 | ⃝ | ⃝ | ⨉ |
    +---+---+---+---+
    
It is a draw
